In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)

# Categorical Variables

# Numerical Variables

In [6]:
usaid = pd.read_csv("USAID_GHSC-PSM_Health_Commodity_Delivery_Dataset_20240105.csv", header= 0, low_memory= False)
usaid = usaid[["Estimated Lead Time in Days", "Illustrative Price", "Order Cycle Time", 
               "Average Days Late", "Ordered Quantity", "Shipped Quantity", "Base Unit Multiplier"]]
usaid.dropna(inplace= True)

# usaid["Ordered Quantity Total"] = usaid["Ordered Quantity"] * usaid["Base Unit Multiplier"]
usaid["Shipped Quantity Total"] = usaid["Shipped Quantity"] * usaid["Base Unit Multiplier"]
usaid.drop(["Ordered Quantity", "Base Unit Multiplier"], axis= 1, inplace= True)

usaid.head(10)
usaid.to_csv("usaid.csv")

,Estimated Lead Time in Days,Illustrative Price,Order Cycle Time,Average Days Late,Shipped Quantity,Shipped Quantity Total
28,161.0,518.58,168,-3.0,1500,72000.0
29,140.0,518.58,142,-1.0,900,43200.0
38,182.0,55.00,186,1.0,110,47520.0
39,154.0,55.00,151,-6.0,198,85536.0
42,105.0,130.00,157,-5.0,15,600.0
44,154.0,260.00,151,-6.0,306,306.0
47,126.0,34.00,164,-2.0,12,240.0
49,84.0,37.08,155,-2.0,200,200.0
50,77.0,19.15,110,6.0,191,191.0
53,98.0,30.00,107,-3.0,260,1326.0
